In [1]:
# https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
pip install -U langchain openai chromadb langchainhub bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.9 MB/s eta 0:00:00
  Using cached pydantic-2.5.2-py3-none-any.whl.metadata (65 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 16.1 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.1 MB/s eta 0:00:00
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached flatbuffers-23.5.26-py2.py3-no

In [6]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [8]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 7.9 MB/s eta 0:00:00
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 

In [9]:
#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [10]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [16]:
# save to disk
#vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db")

# load from disk
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db")

In [4]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("What is Task Decomposition?")

APIConnectionError: Connection error.

## Step 1. Load

In [10]:
len(docs)

1

In [11]:
len(docs[0].page_content)

42824

In [9]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [8]:
print(docs[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

## Step 2. Split

In [11]:
len(splits)

68

In [13]:
len(splits[0].page_content)

654

In [18]:
splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'title': "LLM Powered Autonomous Agents | Lil'Log",
 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:',
 'language': 'en'}

## Step 4. Retrieve

In [20]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [24]:
retrieved_docs = retriever.get_relevant_documents(
    "What are the approaches to Task Decomposition?"
)

In [25]:
len(retrieved_docs)

6

In [38]:
retrieved_docs[0].page_content

'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

In [41]:
retrieved_docs[2].page_content

'Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'

## Step 5. Generate

In [ ]:
# https://smith.langchain.com/hub/rlm/rag-prompt

In [28]:
print(
    prompt.invoke(
        {"context": "filler context", "question": "filler question"}
    ).to_string()
)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [29]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done through prompting techniques like Chain of Thought (CoT) or Tree of Thoughts, as well as task-specific instructions or human inputs. Task decomposition helps agents plan and execute complicated tasks more effectively.

In [30]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done through prompting the model to think step by step or by providing task-specific instructions. Task decomposition helps in enhancing model performance and understanding the model's thinking process. Thanks for asking!"

In [31]:
from operator import itemgetter

from langchain.schema.runnable import RunnableParallel

rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke("What is Task Decomposition")

{'documents': [{'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:',
   'language': 'en',
   'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'title': "LLM Powered Autonomous Agents | Lil'Log"},
  {'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be fra

In [32]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [33]:
from langchain.schema.messages import AIMessage, HumanMessage

condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

'What is the definition of "large" in the context of a language model?'

In [34]:
condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "How do transformers work",
    }
)

'How do transformer models function?'

In [35]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [36]:
chat_history = []

question = "What is Task Decomposition?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What are common ways of doing it?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='Common ways of task decomposition include:\n\n1. Using Chain of Thought (CoT): CoT is a prompting technique where the model is instructed to "think step by step" and decompose complex tasks into smaller and simpler steps. This technique utilizes more test-time computation and helps the model understand and interpret its thinking process.\n\n2. Task-specific instructions: For certain tasks, specific instructions can be provided to guide the task decomposition process. For example, in the context of writing a novel, the instruction "Write a story outline" can help break down the task into subgoals and steps.\n\n3. Human inputs: Task decomposition can also be done with the help of human inputs. Humans can provide guidance, expertise, or domain-specific knowledge to assist in breaking down the task into manageable subtasks or steps.')

In [ ]:
# cleanup
vectorstore.delete_collection()

In [10]:
loader = WebBaseLoader(
    web_paths=("https://namu.wiki/w/%EC%8B%A4%EB%A6%AC%EC%BD%98%EB%B0%B8%EB%A6%AC%20%EC%9D%80%ED%96%89%20%ED%8C%8C%EC%82%B0",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
#            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [11]:
print(docs[0].page_content)

실리콘밸리 은행 파산 - 나무위키  최근 변경  최근 토론  특수 기능  게시판   작성이 필요한 문서  고립된 문서  분류가 되지 않은 문서  편집된 지 오래된 문서  내용이 짧은 문서  내용이 긴 문서  차단 내역  RandomPage  파일 올리기  라이선스   210.211.88.240 Please login!  설정   내 문서 기여 목록 내 토론 기여 목록  로그인          12  역링크 토론 편집 역사  ACL  실리콘밸리 은행 파산  최근 수정 시각: 2023-11-05 16:22:24          2023년/사건사고미국의 사건사고금융 사건 사고       주의. 사건·사고 관련 내용을 설명합니다. 이 문서는 실제로 일어난 사건·사고의 자세한 내용과 설명을 포함하고 있습니다. 국내외 주요 경제/금융 위기 [ 펼치기 · 접기 ]1929 ~ 1940년대대공황 Great Depression2008 ~ 2010년대대침체Great Recession2020 ~ 2023대봉쇄 Great  Lockdown금융 | 경제 | 제2차 세계 대전대침체가 정확히 언제 끝났는지는 경제학자나 역사학자들마다 의견이 다르다.20세기 이전(연준 이전)네덜란드 튤립 투기 파동(1636) · 미시시피 거품(1718) · 남해회사 거품 사태(1720) · 1792년 공황(1792)· 1819년 공황(1819) · 1837년 공황(1837) · 1857년 공황(1857) · 검은 금요일(1869) · 당백전(1866) · 장기불황(1873-1896) · 1907년 은행 패닉(1907)20세기(연준 이후)바이마르 공화국 초인플레이션(1919) · 1929년 월가 대폭락(1929) · 대공황(1929-1939) · 쇼와불황(1930) · 석유 파동(1973-1979) · 검은 월요일(1987) · 일본 거품경제 붕괴(1985-) · 닷컴 버블(1995-2001) · 1997년 외환 위기(1997-2001)2000년대2002년 가계 신용카드 대출 부실 사태(2002) · 서브프라

In [12]:
# save to disk
#vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db")

# load from disk
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_svb")

In [13]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db_svb")
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 요약해 줘")

'실리콘밸리 은행은 파산하였으며, 퍼스트리퍼블릭 은행이 파산 후 JP모건 체이스에게 인수되었습니다. SVB의 파산은 금융업 전체로의 위기로 번지지는 않을 것으로 전망되었습니다.'

In [16]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 자세히 설명해 줘")

'실리콘밸리 은행 파산 사건은 SVB(Silicon Valley Bank)의 파산으로 인해 발생하였습니다. 이 사건은 다른 은행들에 큰 영향을 미치지 않았으며, 금융업계 및 정부 경제 당국에서는 금융업 전체로의 위기로 번지지는 않을 것으로 전망되었습니다. SVB의 파산은 단기적으로 금융시장에 충격을 주었지만, 다른 은행들은 이 사태와는 별개로 운영을 계속할 수 있었습니다.'